In [5]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 92.3MB/s]


In [6]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
df=pd.read_csv('preprocessed_data.csv')#reading the file preprocessed_data.csv

In [8]:
df.head(4)#visulazing the DataFrame


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [9]:
def preprocess(x):#for removing the last character
  x=x[:-1]
  return x

In [10]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [11]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [12]:
df.shape

(2000, 2)

In [13]:
df=df[df['source'].apply(len)<170]#removing sentences where source sentence is greater than 170
df=df[df['target'].apply(len)<200]#removing snetences where target sentence is greater than 200

In [14]:
df.shape#printing the shape

(1990, 2)

In [15]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting in the data in the ratio of 99:1
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


Target:

In [16]:
target_tokenizer= Tokenizer(filters=None,char_level=True,lower=False)#tokenzing the target in character level
target_tokenizer.fit_on_texts(y_train)#fitting on the target train
target_vocab_size= len(target_tokenizer.word_index) + 1
print(len(target_tokenizer.word_index))#printing the vocabulary size

90


In [17]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting target train into sequence
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting target test into sequence

target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding target train 
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding target test

Source:

In [18]:
source_tokenizer= Tokenizer(char_level=True,lower=False)#tokenzing the source in character level
source_tokenizer.fit_on_texts(X_train)#fitting on the source train
source_vocab_size= len(source_tokenizer.word_index) + 1
print(len(source_tokenizer.word_index))#printing the vocabulary size

103


In [19]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting source train into sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting source train into sequence

source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding source train 
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding source test 

In [20]:
#we are reshaping because sparse_categorical_entropy expects 3dimensions
target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [21]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 199, 1)
(20, 199, 1)


In [22]:
#we are reshaping because sparse_categorical_entropy expects 3dimensions
source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [23]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 199, 1)
(20, 199, 1)


In [49]:
X_train.to_csv('X_train2.csv')
y_train.to_csv('y_train2.csv')
X_test.to_csv('X_test2.csv')
y_test.to_csv('y_test2.csv')


In [50]:
import pandas as pd 
pd.DataFrame(source_encoded_docs_train).to_csv("source_encoded_docs_train2.csv")
pd.DataFrame(source_encoded_docs_test).to_csv("source_encoded_docs_test2.csv")
pd.DataFrame(target_encoded_docs_train).to_csv("target_encoded_docs_train2.csv")
pd.DataFrame(target_encoded_docs_test).to_csv("target_encoded_docs_test2.csv")


Model1:

In [24]:
input=tf.keras.layers.Input(shape=(199,))
embed=tf.keras.layers.Embedding(source_vocab_size,256, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.LSTM(128, return_sequences=True)(embed)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512, activation='relu'))(lstm1)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 199)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 199, 256)          26624     
_________________________________________________________________
lstm (LSTM)                  (None, 199, 128)          197120    
_________________________________________________________________
time_distributed (TimeDistri (None, 199, 512)          66048     
_________________________________________________________________
dropout (Dropout)            (None, 199, 512)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 199, 91)           46683     
Total params: 336,475
Trainable params: 336,475
Non-trainable params: 0
_______________________________________________________

In [25]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 9s 648ms/step - loss: 4.0295 - accuracy: 0.3307 - val_loss: 3.0509 - val_accuracy: 0.6756
Epoch 2/100
2/2 [==============================] - 0s 252ms/step - loss: 2.5630 - accuracy: 0.6698 - val_loss: 1.3701 - val_accuracy: 0.7176
Epoch 3/100
2/2 [==============================] - 0s 250ms/step - loss: 1.5293 - accuracy: 0.6910 - val_loss: 1.2024 - val_accuracy: 0.6879
Epoch 4/100
2/2 [==============================] - 0s 246ms/step - loss: 1.3614 - accuracy: 0.6641 - val_loss: 1.0989 - val_accuracy: 0.7291
Epoch 5/100
2/2 [==============================] - 0s 245ms/step - loss: 1.3344 - accuracy: 0.6993 - val_loss: 1.0950 - val_accuracy: 0.7302
Epoch 6/100
2/2 [==============================] - 0s 244ms/step - loss: 1.3359 - accuracy: 0.6934 - val_loss: 1.0957 - val_accuracy: 0.7302
Epoch 7/100
2/2 [==============================] - 0s 243ms/step - loss: 1.3253 - accuracy: 0.6955 - val_loss: 1.0986 - val_accuracy: 0.7302
Epoch 8/100
2

In [28]:
x=model.predict(source_padded_docs_test[:1])[0]

In [29]:
index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'
''.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])


'Photo page..                       <PAD><PAD>   k                    <PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [30]:
print(y_test[:1])

1108    Photo page. You mean the website. OK, I'll go ...
Name: target, dtype: object


In [31]:
X_test[:1]

1108    Photo page... U mean e website huh... Kk, i'll...
Name: source, dtype: object

In [33]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=''.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Photo page... U mean e website huh... Kk, i'll go mail u now...
Actual Output: 
Photo page. You mean the website. OK, I'll go to mail you now.
Predicted Output: 
Photo page..                       <PAD><PAD>   k                    <PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [34]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if '<' in i:
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.3839817133079349, 0, 0.3375804740497263, 0.15042653060571137, 0.46173663094410267, 0.4854917717073234, 0.3976353643835253, 0.5946035575013605, 0, 0.5081327481546147, 0.3549481056010053, 0.3138995505196357, 0.42728700639623407, 0.25650569096216347, 0, 0.22679164443904004, 0.3672056269893592, 0.4172261448611506, 0.43012508513132625, 0.3354232998654124]
The Average Bleu Score is:  0.32245004727098137


Model2:

In [35]:
input=tf.keras.layers.Input(shape=(199,))
embed=tf.keras.layers.Embedding(source_vocab_size,256, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512,activation='relu'))(lstm1)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 199)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 199, 256)          26624     
_________________________________________________________________
bidirectional (Bidirectional (None, 199, 256)          394240    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 199, 512)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 199, 512)          0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 199, 91)           46683     
Total params: 599,131
Trainable params: 599,131
Non-trainable params: 0
_____________________________________________________

In [36]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [37]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 4s 989ms/step - loss: 3.9742 - accuracy: 0.3335 - val_loss: 4.5931 - val_accuracy: 0.6756
Epoch 2/100
2/2 [==============================] - 1s 369ms/step - loss: 3.5696 - accuracy: 0.6468 - val_loss: 1.9741 - val_accuracy: 0.6161
Epoch 3/100
2/2 [==============================] - 1s 375ms/step - loss: 1.9752 - accuracy: 0.5189 - val_loss: 1.2263 - val_accuracy: 0.6955
Epoch 4/100
2/2 [==============================] - 1s 367ms/step - loss: 1.4786 - accuracy: 0.6602 - val_loss: 1.1741 - val_accuracy: 0.6952
Epoch 5/100
2/2 [==============================] - 1s 370ms/step - loss: 1.4346 - accuracy: 0.6614 - val_loss: 1.1194 - val_accuracy: 0.7286
Epoch 6/100
2/2 [==============================] - 1s 364ms/step - loss: 1.3858 - accuracy: 0.6898 - val_loss: 1.0952 - val_accuracy: 0.7281
Epoch 7/100
2/2 [==============================] - 1s 368ms/step - loss: 1.3693 - accuracy: 0.6989 - val_loss: 1.0850 - val_accuracy: 0.7302
Epoch 8/100
2

In [38]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 1s 394ms/step - loss: 1.1057 - accuracy: 0.7160 - val_loss: 0.9645 - val_accuracy: 0.7460
Epoch 2/100
2/2 [==============================] - 1s 368ms/step - loss: 1.1036 - accuracy: 0.7166 - val_loss: 0.9765 - val_accuracy: 0.7462
Epoch 3/100
2/2 [==============================] - 1s 368ms/step - loss: 1.1021 - accuracy: 0.7174 - val_loss: 0.9615 - val_accuracy: 0.7487
Epoch 4/100
2/2 [==============================] - 1s 368ms/step - loss: 1.0997 - accuracy: 0.7174 - val_loss: 0.9702 - val_accuracy: 0.7455
Epoch 5/100
2/2 [==============================] - 1s 370ms/step - loss: 1.0990 - accuracy: 0.7170 - val_loss: 0.9556 - val_accuracy: 0.7467
Epoch 6/100
2/2 [==============================] - 1s 372ms/step - loss: 1.0960 - accuracy: 0.7182 - val_loss: 0.9704 - val_accuracy: 0.7475
Epoch 7/100
2/2 [==============================] - 1s 377ms/step - loss: 1.0952 - accuracy: 0.7189 - val_loss: 0.9628 - val_accuracy: 0.7482
Epoch 8/100
2

In [43]:
model.save_weights('model2.h5')
x=model.predict(source_padded_docs_test[:1])[0]

In [44]:
index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'
''.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])


'Photo page.          ee e              k            <PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [45]:
print(y_test[:1])

1108    Photo page. You mean the website. OK, I'll go ...
Name: target, dtype: object


In [46]:
X_test[:1]

1108    Photo page... U mean e website huh... Kk, i'll...
Name: source, dtype: object

In [47]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=''.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Photo page... U mean e website huh... Kk, i'll go mail u now...
Actual Output: 
Photo page. You mean the website. OK, I'll go to mail you now.
Predicted Output: 
Photo page.          ee e              k            <PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [48]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if '<' in i:
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.1942053406068837, 0, 0, 0.29456425448249246, 0.4924790605054523, 0.4854917717073234, 0.2086130724305753, 0.4428500142691474, 0.3508439695638686, 0.5266403878479265, 0.35782241396102615, 0.31947155212313627, 0.5491004867761125, 0.25650569096216347, 0, 0.2928298013714698, 0.36889397323344053, 0, 0.6431870218238024, 0]
The Average Bleu Score is:  0.289174940583241
